<a href="https://colab.research.google.com/github/NgPcAnhh/Descriptive-statistics-Correlation-of-EPL-strikers-nationality-and-their-performance/blob/main/stat_regression_kiemdinh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beautifulsoup4

In [2]:
# crawl dataa
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
import numpy as np

# xử lý dữ liệu
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [18]:
# kết nối tới drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
results = []

# Crawl từ page 0 đến 19
for page_num in range(42):
    # Tạo URL với số trang hiện tại
    current_url = f"https://footballapi.pulselive.com/football/stats/ranked/players/goals?page={page_num}&pageSize=10&comps=1&compCodeForActivePlayer=EN_PR&positions=FORWARD&altIds=true"

    print(f"\nCrawling page {page_num}...")
    print(f"URL: {current_url}")

    # Gửi request và lấy data
    response = requests.get(current_url)
    data = response.json()

    # Lấy content từ JSON response
    content = data["stats"]["content"]

    for item in content:
        try:
            owner = item["owner"]
            name = owner["name"]["display"]
            id = int(owner["id"])

            # Try to get country, fall back to isoCode if country is not available
            try:
                country = owner["nationalTeam"]["country"]
            except (KeyError, TypeError):
                try:
                    country = owner["nationalTeam"]["isoCode"]
                except (KeyError, TypeError):
                    # Skip this item if both country and isoCode are missing
                    print(f"Skipped item: Missing country information for {owner.get('name', {}).get('display', 'Unknown player')}")
                    continue

            # Thêm vào kết quả
            player_info = {
                "name": name,
                "country": country,
                "id": id
            }
            results.append(player_info)
            print(f"Added: {player_info}")
        except Exception as e:
            print(f"Error processing item: {e}")
            continue
            print(f"Added: {player_info}")


# In kết quả cuối cùng
print("\nFinal Results:")
print(json.dumps(results, indent=2))


Crawling page 0...
URL: https://footballapi.pulselive.com/football/stats/ranked/players/goals?page=0&pageSize=10&comps=1&compCodeForActivePlayer=EN_PR&positions=FORWARD&altIds=true
Added: {'name': 'Alan Shearer', 'country': 'England', 'id': 89}
Added: {'name': 'Harry Kane', 'country': 'England', 'id': 3960}
Added: {'name': 'Wayne Rooney', 'country': 'England', 'id': 2064}
Added: {'name': 'Andrew Cole', 'country': 'England', 'id': 725}
Added: {'name': 'Sergio Agüero', 'country': 'Argentina', 'id': 4328}
Added: {'name': 'Mohamed Salah', 'country': 'Egypt', 'id': 5178}
Added: {'name': 'Thierry Henry', 'country': 'France', 'id': 1659}
Added: {'name': 'Robbie Fowler', 'country': 'England', 'id': 277}
Added: {'name': 'Jermain Defoe', 'country': 'England', 'id': 1526}
Added: {'name': 'Michael Owen', 'country': 'England', 'id': 1208}

Crawling page 1...
URL: https://footballapi.pulselive.com/football/stats/ranked/players/goals?page=1&pageSize=10&comps=1&compCodeForActivePlayer=EN_PR&positions

In [4]:
players_full_stats = []

for player in results:
    player_id = player["id"]
    player_stats_url = f'https://www.premierleague.com/players/{player_id}/player/stats'

    print(f"\nCrawling stats for player ID {player_id}...")
    print(f"URL: {player_stats_url}")

    try:
        response = requests.get(player_stats_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Thu thập stats và loại bỏ khoảng trắng, ký tự xuống dòng
        stats = {
            "id": player_id,
            "name": player["name"],
            "country": player["country"],
            "goals": soup.find('span', class_='statgoals').text.strip() if soup.find('span', class_='statgoals') else "N/A",
            "goals_per_match": soup.find('span', class_='statgoals_per_game').text.strip() if soup.find('span', class_='statgoals_per_game') else "N/A",
            "headed_goals": soup.find('span', class_='statatt_hd_goal').text.strip() if soup.find('span', class_='statatt_hd_goal') else "N/A",
            "shooting_accuracy": soup.find('span', class_='statshot_accuracy').text.strip() if soup.find('span', class_='statshot_accuracy') else "N/A",
            "hit_woodwork": soup.find('span', class_='stathit_woodwork').text.strip() if soup.find('span', class_='stathit_woodwork') else "N/A",
            "big_chances_missed": soup.find('span', class_='statbig_chance_missed').text.strip() if soup.find('span', class_='statbig_chance_missed') else "N/A",
            "assists": soup.find('span', class_='statgoal_assist').text.strip() if soup.find('span', class_='statgoal_assist') else "N/A",
            "big_chances_created": soup.find('span', class_='statbig_chance_created').text.strip() if soup.find('span', class_='statbig_chance_created') else "N/A"
        }

        # Thêm vào danh sách kết quả
        players_full_stats.append(stats)
        print(f"Added stats for: {player['name']}")

    except Exception as e:
        print(f"Error crawling stats for player {player_id}: {str(e)}")
        continue

# In kết quả cuối cùng
print("\nFinal Results with Full Stats:")
print(json.dumps(players_full_stats, indent=2))
print(f"\nTotal players processed: {len(players_full_stats)}")

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
    "hit_woodwork": "1",
    "big_chances_missed": "0",
    "assists": "37",
    "big_chances_created": "0"
  },
  {
    "id": 4481,
    "name": "Olivier Giroud",
    "country": "France",
    "goals": "90",
    "goals_per_match": "0.35",
    "headed_goals": "32",
    "shooting_accuracy": "39%",
    "hit_woodwork": "20",
    "big_chances_missed": "88",
    "assists": "28",
    "big_chances_created": "41"
  },
  {
    "id": 1249,
    "name": "Kevin Davies",
    "country": "England",
    "goals": "88",
    "goals_per_match": "0.2",
    "headed_goals": "15",
    "shooting_accuracy": "37%",
    "hit_woodwork": "6",
    "big_chances_missed": "14",
    "assists": "55",
    "big_chances_created": "10"
  },
  {
    "id": 8454,
    "name": "Callum Wilson",
    "country": "England",
    "goals": "88",
    "goals_per_match": "0.38",
    "headed_goals": "15",
    "shooting_accuracy": "44%",
    "hit_woodwork": "13",
    "big_chances_missed": 

In [15]:
import pandas as pd

# Hiển thị toàn bộ cột và dòng
pd.set_option('display.max_columns', None)  # Không giới hạn số cột
pd.set_option('display.max_rows', None)     # Không giới hạn số dòng

# Điều chỉnh chiều rộng tối đa để sử dụng toàn bộ không gian
pd.set_option('display.width', 200)         # Tăng chiều rộng tối đa

In [114]:
import pandas as pd
import numpy as np

# Giả định players_full_stats là danh sách các thông tin cầu thủ đã được crawl từ web
# Tạo DataFrame từ dữ liệu được crawl
df = pd.DataFrame(players_full_stats)

# Tạo bản sao để không ảnh hưởng đến df gốc
df_processed = df.copy()

# Danh sách các cột cần xử lý
columns_to_process = [
    'headed_goals',
    'shooting_accuracy',
    'hit_woodwork',
    'big_chances_missed',
    'big_chances_created'
]

# Xử lý từng cột
for col in columns_to_process:
    # Kiểm tra kiểu dữ liệu của cột
    if df_processed[col].dtype == 'object':  # Nếu cột là kiểu string
        # Xử lý cột có chứa '%'
        df_processed[col] = df_processed[col].apply(lambda x: str(x).replace('%', '') if isinstance(x, str) else x)
        df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')

    # Tính giá trị trung bình (bỏ qua giá trị 0 và NA)
    mean_value = df_processed[df_processed[col] > 0][col].mean()

    # Thay thế giá trị 0 và NA bằng mean
    df_processed.loc[df_processed[col].isin([0, np.nan]), col] = mean_value

    # Làm tròn và định dạng
    if col == 'shooting_accuracy':
        df_processed[col] = df_processed[col].round(2).astype(str) + '%'
    else:
        df_processed[col] = df_processed[col].round(2)

df_processed = df_processed.head(413)  # Lọc dữ liệu

# In kết quả
print("\nDữ liệu sau khi xử lý:")
print(df_processed)

# Lưu dữ liệu đã xử lý vào file CSV
save_path = '/content/drive/My Drive/Kinh tế lượng/dataset/stats_EPL.csv'
df_processed.to_csv(save_path, index=False, encoding='utf-8')

print(f"Dữ liệu đã được lưu vào file CSV tại: {save_path}")


Dữ liệu sau khi xử lý:
         id                       name               country goals goals_per_match  headed_goals shooting_accuracy  hit_woodwork  big_chances_missed assists  big_chances_created
0        89               Alan Shearer               England   260            0.59          6.67            100.0%          7.21               27.52      64                18.83
1      3960                 Harry Kane               England   213            0.67         40.00             44.0%         36.00              128.00      46                81.00
2      2064               Wayne Rooney               England   208            0.42         21.00             38.0%         29.00               54.00     103                65.00
3       725                Andrew Cole               England   187            0.45          6.67             41.0%          7.21               27.52      73                18.83
4      4328              Sergio Agüero             Argentina   184            0.67    

xong bước crawl và xử lý data


In [100]:
df = pd.read_csv('/content/drive/My Drive/Kinh tế lượng/dataset/stats_EPL.csv')

In [101]:
df.head()

,id,name,country,goals,goals_per_match,headed_goals,shooting_accuracy,hit_woodwork,big_chances_missed,assists,big_chances_created
0,89,Alan Shearer,England,260.0,0.59,6.67,100.0%,7.21,27.52,64.0,18.83
1,3960,Harry Kane,England,213.0,0.67,40.00,44.0%,36.00,128.00,46.0,81.00
2,2064,Wayne Rooney,England,208.0,0.42,21.00,38.0%,29.00,54.00,103.0,65.00
3,725,Andrew Cole,England,187.0,0.45,6.67,41.0%,7.21,27.52,73.0,18.83
4,4328,Sergio Agüero,Argentina,184.0,0.67,19.00,42.0%,34.00,128.00,47.0,62.00


In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413 entries, 0 to 412
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   413 non-null    int64  
 1   name                 413 non-null    object 
 2   country              413 non-null    object 
 3   goals                405 non-null    float64
 4   goals_per_match      405 non-null    float64
 5   headed_goals         413 non-null    float64
 6   shooting_accuracy    413 non-null    object 
 7   hit_woodwork         413 non-null    float64
 8   big_chances_missed   413 non-null    float64
 9   assists              405 non-null    float64
 10  big_chances_created  413 non-null    float64
dtypes: float64(7), int64(1), object(3)
memory usage: 35.6+ KB


In [103]:
# xử lý chuyển đổi object của shooting_accuracy
df['shooting_accuracy'] = df['shooting_accuracy'].str.rstrip('%').astype('float') / 100.0

In [104]:
# xóa đi cột name và country của cầu thủ
df = df.drop(['name', 'country'], axis=1)

In [68]:
df.head()

,id,goals,goals_per_match,headed_goals,shooting_accuracy,hit_woodwork,big_chances_missed,assists,big_chances_created
0,89,260.0,0.59,6.67,1.00,7.21,27.52,64.0,18.83
1,3960,213.0,0.67,40.00,0.44,36.00,128.00,46.0,81.00
2,2064,208.0,0.42,21.00,0.38,29.00,54.00,103.0,65.00
3,725,187.0,0.45,6.67,0.41,7.21,27.52,73.0,18.83
4,4328,184.0,0.67,19.00,0.42,34.00,128.00,47.0,62.00


In [106]:
df.head()

,id,goals,goals_per_match,headed_goals,shooting_accuracy,hit_woodwork,big_chances_missed,assists,big_chances_created
0,89,260.0,0.59,6.67,1.00,7.21,27.52,64.0,18.83
1,3960,213.0,0.67,40.00,0.44,36.00,128.00,46.0,81.00
2,2064,208.0,0.42,21.00,0.38,29.00,54.00,103.0,65.00
3,725,187.0,0.45,6.67,0.41,7.21,27.52,73.0,18.83
4,4328,184.0,0.67,19.00,0.42,34.00,128.00,47.0,62.00


# Thống kê

tabstat var1 var2 var3 , statistics(mean, count, max, min, range, sd, variance, cv, semean, skewness, kurtosis, median)


In [70]:
df.describe()

,id,goals,goals_per_match,headed_goals,shooting_accuracy,hit_woodwork,big_chances_missed,assists,big_chances_created
count,413.000000,405.000000,405.000000,413.000000,413.000000,413.000000,413.000000,405.000000,413.000000
mean,8169.723971,39.797531,0.257778,6.684237,0.402712,7.252639,27.579758,16.167901,18.857748
std,17285.858496,37.476706,0.110092,5.218083,0.091410,5.450223,20.207641,16.106590,15.171647
min,25.000000,10.000000,0.060000,1.000000,0.200000,1.000000,1.000000,0.000000,1.000000
25%,1240.000000,15.000000,0.180000,4.000000,0.370000,4.000000,16.000000,6.000000,10.000000
50%,2861.000000,25.000000,0.240000,6.670000,0.402000,7.210000,27.520000,10.000000,18.830000
75%,5762.000000,51.000000,0.310000,6.670000,0.410000,7.210000,27.520000,21.000000,18.830000
max,134048.000000,260.000000,0.890000,40.000000,1.000000,44.000000,146.000000,103.000000,129.000000


In [71]:
from scipy.stats import skew, kurtosis

# Định nghĩa hàm thống kê
def calculate_statistics(df):
    statistics = {}
    for col in df.columns:
        stats = {
            'range': np.max(df[col]) - np.min(df[col]),
            'sd': np.std(df[col], ddof=1),
            'variance': np.var(df[col], ddof=1),
            'cv': np.std(df[col], ddof=1) / np.mean(df[col]) if np.mean(df[col]) != 0 else None,
            'semean': np.std(df[col], ddof=1) / np.sqrt(df[col].count()),
            'skewness': skew(df[col], nan_policy='omit'),
            'kurtosis': kurtosis(df[col], nan_policy='omit'),
            'median': np.median(df[col])
        }
        statistics[col] = stats
    return statistics

# Lọc cột theo danh sách 'data' và tính thống kê
filtered_data = df
statistics_result = calculate_statistics(filtered_data)

# Chuyển đổi kết quả thành DataFrame
stats_df = pd.DataFrame(statistics_result).T  # .T để chuyển vị, hiển thị cột thống kê dọc
stats_df.index.name = "Column"
stats_df.reset_index(inplace=True)  # Đặt tên cột cho index

# Hiển thị kết quả
print(stats_df)


                Column      range            sd      variance        cv      semean  skewness   kurtosis    median
0                   id  134023.00  17285.858496  2.988009e+08  2.115844  850.581490  4.462654  22.897234  2861.000
1                goals     250.00     37.476706  1.404503e+03  0.941684    1.862232  2.298915   6.507120       NaN
2      goals_per_match       0.83      0.110092  1.212030e-02  0.427082    0.005471  1.485842   4.083383       NaN
3         headed_goals      39.00      5.218083  2.722839e+01  0.780655    0.256765  2.787078  11.600571     6.670
4    shooting_accuracy       0.80      0.091410  8.355735e-03  0.226985    0.004498  4.682949  28.519887     0.402
5         hit_woodwork      43.00      5.450223  2.970493e+01  0.751481    0.268188  2.488046   9.675387     7.210
6   big_chances_missed     145.00     20.207641  4.083488e+02  0.732698    0.994353  2.488976   8.761089    27.520
7              assists     103.00     16.106590  2.594222e+02  0.996208    0.800

tabulate var1

In [72]:
# Tính tần suất cho cột 'goals'
def calculate_frequency(df, column):
    frequency_table = df[column].value_counts().sort_index()  # Sắp xếp theo giá trị
    total_count = frequency_table.sum()

    # Tạo bảng thống kê
    stats = pd.DataFrame({
        'Unique Value': frequency_table.index,
        'Frequency': frequency_table.values,
        'Percentage (%)': (frequency_table / total_count * 100).round(2),
        'Cumulative Percentage (%)': (frequency_table.cumsum() / total_count * 100).round(2)
    })

    return stats

# Ví dụ áp dụng với dataset
frequency_stats = calculate_frequency(df, 'goals')
print(frequency_stats)


       Unique Value  Frequency  Percentage (%)  Cumulative Percentage (%)
goals                                                                    
10.0           10.0         31            7.65                       7.65
11.0           11.0         19            4.69                      12.35
12.0           12.0         13            3.21                      15.56
13.0           13.0         18            4.44                      20.00
14.0           14.0         13            3.21                      23.21
15.0           15.0          8            1.98                      25.19
16.0           16.0          8            1.98                      27.16
17.0           17.0         13            3.21                      30.37
18.0           18.0          9            2.22                      32.59
19.0           19.0         12            2.96                      35.56
20.0           20.0         14            3.46                      39.01
21.0           21.0          8        

tabstat var1 var2 var3 var4 var 5, statistics(mean, count, max, min, range, sd, variance, cv, semean, skewness, kurtosis, median) by(var0)


In [80]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis

# Phân nhóm goals thành bốn loại
df['Goals_Group'] = pd.cut(df['goals'], bins=[-float('inf'), 50, 100, 200, float('inf')],
                           labels=['Dưới 50', 'Trên 50', 'Trên 100', 'Trên 200'])

# Định nghĩa hàm tính thống kê mô tả theo nhóm
def calculate_group_statistics(df, group_column, target_columns):
    grouped_stats = []
    for group, group_data in df.groupby(group_column):
        stats = {'Group': group}
        for col in target_columns:
            stats[f'{col}_mean'] = group_data[col].mean()
            stats[f'{col}_count'] = group_data[col].count()
            stats[f'{col}_max'] = group_data[col].max()
            stats[f'{col}_min'] = group_data[col].min()
            stats[f'{col}_range'] = group_data[col].max() - group_data[col].min()
            stats[f'{col}_sd'] = group_data[col].std(ddof=1)
            stats[f'{col}_variance'] = group_data[col].var(ddof=1)
            stats[f'{col}_cv'] = group_data[col].std(ddof=1) / group_data[col].mean() if group_data[col].mean() != 0 else None
            stats[f'{col}_semean'] = group_data[col].std(ddof=1) / np.sqrt(group_data[col].count())
            stats[f'{col}_skewness'] = skew(group_data[col], nan_policy='omit')
            stats[f'{col}_kurtosis'] = kurtosis(group_data[col], nan_policy='omit')
            stats[f'{col}_median'] = group_data[col].median()
        grouped_stats.append(stats)
    return pd.DataFrame(grouped_stats)

# Áp dụng hàm cho nhóm Goals_Group
group_column = 'Goals_Group'  # Phân nhóm theo Goals_Group
target_columns = ['goals_per_match']

# Tính thống kê
grouped_statistics = calculate_group_statistics(df, group_column, target_columns)

# Hiển thị kết quả
print(grouped_statistics)


      Group  goals_per_match_mean  goals_per_match_count  goals_per_match_max  goals_per_match_min  goals_per_match_range  goals_per_match_sd  goals_per_match_variance  goals_per_match_cv  \
0   Dưới 50              0.225446                    303                 0.65                 0.06                   0.59            0.082625                  0.006827            0.366496   
1   Trên 50              0.321486                     74                 0.89                 0.16                   0.73            0.110697                  0.012254            0.344330   
2  Trên 100              0.424800                     25                 0.68                 0.21                   0.47            0.115330                  0.013301            0.271492   
3  Trên 200              0.560000                      3                 0.67                 0.42                   0.25            0.127671                  0.016300            0.227985   

   goals_per_match_semean  goals_per_match_s

<ipython-input-80-1a1aa65cd5cc>:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for group, group_data in df.groupby(group_column):


pwcorr GPA Love Study Sport Drink Game, sig


# Hồi quy



In [89]:
import statsmodels.api as sm

# Xử lý dữ liệu để tránh lỗi MissingDataError
def clean_data(X, Y):
    # Loại bỏ các giá trị NaN hoặc vô hạn trong cả X và Y
    data = pd.concat([X, Y], axis=1).dropna()  # Gộp X và Y, loại bỏ hàng có NaN
    data = data.replace([np.inf, -np.inf], np.nan).dropna()  # Loại bỏ giá trị inf
    return data[X.columns], data[Y.name]

# Xác định biến phụ thuộc (Y) và biến độc lập (X)
Y = df['goals']
X = df[['goals_per_match', 'headed_goals', 'shooting_accuracy', 'hit_woodwork', 'big_chances_missed']]

# Làm sạch dữ liệu
X, Y = clean_data(X, Y)

# Thêm cột hằng số (intercept)
X = sm.add_constant(X)

# Hồi quy tuyến tính
model = sm.OLS(Y, X).fit()

# In kết quả giống Stata
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                  goals   R-squared:                       0.539
Model:                            OLS   Adj. R-squared:                  0.533
Method:                 Least Squares   F-statistic:                     93.17
Date:                Thu, 27 Mar 2025   Prob (F-statistic):           8.00e-65
Time:                        15:23:44   Log-Likelihood:                -1885.1
No. Observations:                 405   AIC:                             3782.
Df Residuals:                     399   BIC:                             3806.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                -28.5757      6

# Kiểm định

In [93]:
from statsmodels.stats.diagnostic import linear_reset

# Kiểm định Ramsey RESET
reset_test = linear_reset(model, power=2, use_f=True)
print("RESET Test:")
print(reset_test)

RESET Test:
<F test: F=0.5859324739018885, p=0.44445001063385126, df_denom=398, df_num=1>


In [91]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Tính VIF cho từng biến độc lập
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)


             Variable        VIF
0               const  23.669470
1     goals_per_match   1.251259
2        headed_goals   1.635006
3   shooting_accuracy   1.069491
4        hit_woodwork   2.213858
5  big_chances_missed   2.473264


In [116]:
# Kiểm định Durbin-Watson (estat durbinalt) - Tự tương quan bậc 1

from statsmodels.stats.stattools import durbin_watson

# Tính hệ số Durbin-Watson
dw_stat = durbin_watson(model.resid)
print(f"Durbin-Watson Statistic: {dw_stat:.4f}")

Durbin-Watson Statistic: 0.9368


Ý nghĩa:

DW ≈ 2 → Không có tự tương quan.

DW < 2 → Tự tương quan dương.

DW > 2 → Tự tương quan âm.

In [117]:
#Kiểm định Breusch-Godfrey LM (bgodfrey) - Tự tương quan bậc cao

from statsmodels.stats.diagnostic import acorr_breusch_godfrey

# Kiểm định Breusch-Godfrey bậc 1
bg_test = acorr_breusch_godfrey(model, nlags=1)
print(f"Breusch-Godfrey Test: LM-stat = {bg_test[0]:.4f}, p-value = {bg_test[1]:.4f}")


Breusch-Godfrey Test: LM-stat = 118.9344, p-value = 0.0000


 Ý nghĩa:

p-value < 0.05 → Có tự tương quan.

p-value > 0.05 → Không có tự tương quan.

In [118]:
#  Kiểm định White (estat imtest) - Phương sai sai số thay đổi

from statsmodels.stats.diagnostic import het_white

# Kiểm định White
white_test = het_white(model.resid, model.model.exog)
print(f"White Test: LM-stat = {white_test[0]:.4f}, p-value = {white_test[1]:.4f}")

White Test: LM-stat = 219.7660, p-value = 0.0000


 Ý nghĩa:

p-value < 0.05 → Phương sai sai số thay đổi (heteroscedasticity).

p-value > 0.05 → Không có phương sai sai số thay đổi.

In [119]:
# Kiểm định Breusch-Pagan (estat hettest) - Phương sai sai số thay đổi

from statsmodels.stats.diagnostic import het_breuschpagan

# Kiểm định Breusch-Pagan
bp_test = het_breuschpagan(model.resid, model.model.exog)
print(f"Breusch-Pagan Test: LM-stat = {bp_test[0]:.4f}, p-value = {bp_test[1]:.4f}")


Breusch-Pagan Test: LM-stat = 105.4880, p-value = 0.0000


→ Ý nghĩa:

p-value < 0.05 → Phương sai sai số thay đổi.

p-value > 0.05 → Không có phương sai sai số thay đổi.